In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
#from keras_tqdm import TQDMNotebookCallback

from keras.callbacks import TensorBoard
from keras import backend as K

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

from datetime import datetime
import os

import numpy as np
import pandas as pd

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'fc_model.h5'

weight_name = 'secend_try_vgg16.h5'

# dimensions of our images.
img_width, img_height = 128, 128

train_data_dir = 'data/data/train'
validation_data_dir = 'data/data/validation'
nb_train_samples = 1600
nb_validation_samples = 695
epochs = 10#50

batch_size = 16

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model

vgg16 = VGG16(weights='imagenet', include_top=False)

#Even I don't know why this works, 
#I think MaxPooling2D() will give me the same result,
#but I failed to train, so start from block5_conv3 ,
# using x = GlobalAveragePooling2D()(x)  instead of MaxPooling.

x = vgg16.get_layer('block5_conv3').output
x = GlobalAveragePooling2D()(x)

#x = Flatten(name='flatten')(x)
x = Dense(256, activation='relu', name='dense_1')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid', name='dense_2')(x)

model = Model(inputs=vgg16.input, outputs=x)

# set the first 10 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False


Using TensorFlow backend.


Found 1600 images belonging to 2 classes.
Found 695 images belonging to 2 classes.


In [7]:
#model.load_weights('secend_try_vgg16.h5')

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
# When using adam 
# optimizer='adam',

model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=0.0001, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

epochs = 1

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose = 1,
    callbacks=[early_stopping])
               
model.save('VGG16_S.h5','w')

Epoch 1/1
  6/100 [>.............................] - ETA: 239s - loss: 0.1044 - acc: 0.9479

KeyboardInterrupt: 

In [2]:
model.load_weights('secend_try_vgg16.h5')

In [12]:

from keras.preprocessing.image import load_img,img_to_array
import numpy as np
img = load_img("data/data/train/1/.jpg",False,target_size=(img_width,img_height))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.

pred1 = model.predict(x)
#pred = model.predict_classes(x)
#prob = model.predict_proba(x)
#print(pred, prob)
pred1

array([[ 1.]], dtype=float32)

In [14]:
test_data_dir = 'data/test1'

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode=None,
    shuffle = False)

preds = model.predict_generator(test_generator, 1531,verbose = 1)
preds_rounded = []

for pred in preds:
    if (pred > .5):
        preds_rounded.append("1")
    else:
        preds_rounded.append("0")
        
preds_filenames = [int(x.replace("tomz/", "").replace(".jpg", "")) for x in test_generator.filenames]   

data = (list(zip(preds_filenames, preds_rounded)))

import pandas as pd
df_result = pd.DataFrame(data, columns=["name", "invasive"])
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)

df_result.to_csv("submission_03.csv", encoding="utf8", index=True)

Found 1531 images belonging to 1 classes.
1531/1531 [==============================] - 296s   


In [15]:
model.save('VGG16_S.h5','w')

In [ ]:
s1 = pd.read_csv("submission_01.csv")
s2 = pd.read_csv("submission_02.csv")
s3 = pd.read_csv("submission_03.csv")
s4 = pd.read_csv("submission_03.csv")

s11 = np.array(s1)
s21 = np.array(s2)
s31 = np.array(s3)
s41 = np.array(s4)

s5 = s11+s21+2*s31+2*s41

s5[s5<2] = 0
s5[s5>0] = 1

s5.to_csv(("submission_05.csv", encoding="utf8", index=True)